In [79]:
# 哈工大社会计算与信息检索研究中心同义词词林扩展版.txt
# 读取文件 只保留B开头的
# 生成一个字典
import re
import json
import os
import sys
out = {}
with open('哈工大社会计算与信息检索研究中心同义词词林扩展版.txt', 'r') as f:
    for line in f:
        if line.startswith('B'):
            line = line.strip()
            line = re.sub(r'\s+', ' ', line)
            line = line.split(' ')
            out[line[0]] = line[1:]
    
# 将KEY的前两位作为新的KEY
out2 = {}
for key in out:
    new_key = key[:2]
    if new_key not in out2:
        out2[new_key] = []
    for i in out[key]:
        out2[new_key].append(i)
        # 将 out2是字典 KEY是类型名字 转化为 X Y变量，X是全部元素，Y是X中元素对于的类型
X = []
Y_label = []
for key in out2:
    for i in out2[key]:
        X.append(i)
        Y_label.append(key)
# Y_label是标签类型，Y是同长度的数组，将标签转化为数字 
Y = []
temp = {}
indexA = 0
for i in Y_label:
    if i not in temp:
        temp[i] = indexA
        indexA += 1
    Y.append(temp[i])
    
# 保存X和Y
with open('X.txt', 'w') as f:
    f.write(json.dumps(X))
with open('Y.txt', 'w') as f:
    f.write(json.dumps(Y))
    


In [80]:
len(X)

18849

In [46]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch.nn.functional as F

import numpy as np
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

import json

import torch



device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
model = model.to(device)


# 将X分为6份
X = []
with open('X.txt', 'r') as f:
    X = json.loads(f.read())
X = np.array(X)
X = np.array_split(X, 6)


for i in range(6):
    temp = X[i]
    temp = temp.tolist()
    input_texts = temp
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

    batch_dict = {k: v.to(device) for k, v in batch_dict.items()}
    outputs = model(**batch_dict)


    def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

    embeddings = F.normalize(embeddings, p=2, dim=1)  #torch.Size([40, 1024])
    text = 'embeddings' + str(i) + '.npy'
    np.save(text, embeddings.cpu().detach().numpy())
    torch.cuda.empty_cache()

In [ ]:
# 读取    text = 'embeddings' + str(i) + '.npy' 
out3 = []
for i in range(6):
    text = 'embeddings' + str(i) + '.npy'
    temp = np.load(text)
    out3.append(temp)
    
# type(out3[0]) numpy.ndarray
# 合并
out4 = np.concatenate(out3, axis=0)
# 保存
np.save('embeddings.npy', out4)
# 读取
out5 = np.load('embeddings.npy')

In [116]:
import numpy as np

# 定义Sigmoid激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 定义Sigmoid激活函数的导数
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

# 定义softmax函数，用于输出层的激活
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

# 定义交叉熵损失函数
def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    log_likelihood = -np.log(y_pred[range(m), y_true])
    loss = np.sum(log_likelihood) / m
    return loss

# 初始化权重和偏置
def init_weights(input_dim, hidden_dim, output_dim):
    W1 = np.random.randn(input_dim, hidden_dim) * 0.01
    b1 = np.zeros((1, hidden_dim))
    W2 = np.random.randn(hidden_dim, output_dim) * 0.01
    b2 = np.zeros((1, output_dim))
    return W1, b1, W2, b2
# 更新权重和偏置
def update_weights(W1, b1, W2, b2, dW1, db1, dW2, db2, lr):
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    return W1, b1, W2, b2


# 定义前向和反向传播过程
def forward_backward_pass(x, y, W1, b1, W2, b2):
    # 前向传播
    z1 = x.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    y_pred = softmax(z2)
    # 计算损失
    loss = cross_entropy_loss(y_pred, y)
    
    # 反向传播
    m = y.shape[0]
    dz2 = y_pred
    dz2[range(m), y] -= 1
    dz2 /= m
    dW2 = a1.T.dot(dz2)
    db2 = np.sum(dz2, axis=0, keepdims=True)
    dz1 = dz2.dot(W2.T) * sigmoid_derivative(z1)
    dW1 = x.T.dot(dz1)
    db1 = np.sum(dz1, axis=0, keepdims=True)
    
    return loss, dW1, db1, dW2, db2



def train(x, y, hidden_dim, output_dim, lr, epochs, batch_size):
    input_dim = x.shape[1]
    W1, b1, W2, b2 = init_weights(input_dim, hidden_dim, output_dim)
    
    # 计算批次数量
    num_batches = x.shape[0] // batch_size
    if x.shape[0] % batch_size != 0:
        num_batches += 1
    
    for i in range(epochs):
        # 打乱顺序
        indices = np.random.permutation(x.shape[0])
        x = x[indices]
        y = y[indices]
        for j in range(num_batches):
            # 获取当前批次的数据
            start = j * batch_size
            end = min((j + 1) * batch_size, x.shape[0])
            x_batch = x[start:end]
            y_batch = y[start:end]
            
            # 前向和反向传播
            loss, dW1, db1, dW2, db2 = forward_backward_pass(x_batch, y_batch, W1, b1, W2, b2)
            
            # 更新权重
            W1, b1, W2, b2 = update_weights(W1, b1, W2, b2, dW1, db1, dW2, db2, lr)
        if i % 10 == 0:
            print(f'Accuracy: {accuracy(x, y, *trained_weights):.4f}')
            print(f'Epoch {i}, Loss: {loss:.4f}')
    return W1, b1, W2, b2

# 示例：使用神经网络进行分类

x = out5 = np.load('000000000.npy')
with open('Y.txt', 'r') as f:
    y = json.loads(f.read())
y = np.array(y)

hidden_dim = 32
output_dim = 18
lr = 0.1
epochs = 1000

trained_weights = train(x, y, hidden_dim, output_dim, lr, epochs, batch_size=32)






Accuracy: 0.1437
Epoch 0, Loss: 3.3305
Accuracy: 0.1437
Epoch 10, Loss: 2.4554
Accuracy: 0.1437
Epoch 20, Loss: 4.5416
Accuracy: 0.1437
Epoch 30, Loss: 2.7787
Accuracy: 0.1437
Epoch 40, Loss: 0.5890
Accuracy: 0.1437
Epoch 50, Loss: 1.8844
Accuracy: 0.1437
Epoch 60, Loss: 0.3902
Accuracy: 0.1437
Epoch 70, Loss: 1.9036
Accuracy: 0.1437
Epoch 80, Loss: 0.0471
Accuracy: 0.1437
Epoch 90, Loss: 0.1776
Accuracy: 0.1437
Epoch 100, Loss: 0.0194
Accuracy: 0.1437
Epoch 110, Loss: 0.0137
Accuracy: 0.1437
Epoch 120, Loss: 0.0673
Accuracy: 0.1437
Epoch 130, Loss: 0.1716
Accuracy: 0.1437
Epoch 140, Loss: 0.0669
Accuracy: 0.1437
Epoch 150, Loss: 0.0557
Accuracy: 0.1437
Epoch 160, Loss: 0.0481
Accuracy: 0.1437
Epoch 170, Loss: 0.1534


KeyboardInterrupt: 

Accuracy: 0.4136 Epoch 960, Loss: 7.4448
Accuracy: 0.4142 Epoch 961, Loss: 7.4386
Accuracy: 0.4148 Epoch 962, Loss: 7.4324
Accuracy: 0.4154 Epoch 963, Loss: 7.4263
Accuracy: 0.4160 Epoch 964, Loss: 7.4201
Accuracy: 0.4166 Epoch 965, Loss: 7.4139
Accuracy: 0.4172 Epoch 966, Loss: 7.4078
Accuracy: 0.4178 Epoch 967, Loss: 7.4016
Accuracy: 0.4184 Epoch 968, Loss: 7.3955
Accuracy: 0.4190 Epoch 969, Loss: 7.3893
Accuracy: 0.4196 Epoch 970, Loss: 7.3832
Accuracy: 0.4202 Epoch 971, Loss: 7.3770
Accuracy: 0.4208 Epoch 972, Loss: 7.3709
Accuracy: 0.4214 Epoch 973, Loss: 7.3647
Accuracy: 0.4220 Epoch 974, Loss: 7.3586
Accuracy: 0.4226 Epoch 975, Loss: 7.3525
Accuracy: 0.4232 Epoch 976, Loss: 7.3463
Accuracy: 0.4238 Epoch 977, Loss: 7.3402
Accuracy: 0.4244 Epoch 978, Loss: 7.3341
Accuracy: 0.4250 Epoch 979, Loss: 7.3280
Accuracy: 0.4256 Epoch 980, Loss: 7.3219
Accuracy: 0.4262 Epoch 981, Loss: 7.3158
Accuracy: 0.4268 Epoch 982, Loss: 7.3097
Accuracy: 0.4274 Epoch 983, Loss: 7.3036
Accuracy: 0.4280 Epoch 984, Loss: 7.2975
Accuracy: 0.4286 Epoch 985, Loss: 7.2914
Accuracy: 0.4292 Epoch 986, Loss: 7.2853
Accuracy: 0.4298 Epoch 987, Loss: 7.2792
Accuracy: 0.4304 Epoch 988, Loss: 7.2731
Accuracy: 0.4310 Epoch 989, Loss: 7.2671
Accuracy: 0.4316 Epoch 990, Loss: 7.2610
Accuracy: 0.4322 Epoch 991, Loss: 7.2549
Accuracy: 0.4328 Epoch 992, Loss: 7.2489
Accuracy: 0.4334 Epoch 993, Loss: 7.2428
Accuracy: 0.4340 Epoch 994, Loss: 7.2368
Accuracy: 0.4346 Epoch 995, Loss: 7.2307
Accuracy: 0.4352 Epoch 996, Loss: 7.2247

In [117]:
# 计算准确率
def accuracy(x, y, W1, b1, W2, b2):
    z1 = x.dot(W1) + b1
    a1 = relu(z1)
    z2 = a1.dot(W2) + b2
    y_pred = np.argmax(softmax(z2), axis=1)
    return np.mean(y_pred == y)
pr
print(f'Accuracy: {accuracy(x, y, *trained_weights):.4f}')


Accuracy: 0.1437
